# Terceiro Projeto: Construindo a Árvore de Sintaxe Abstrata

Uma árvore de sintaxe abstrata (do inglês, *Abstract Syntax Tree* - AST) é uma estrutura de dados que representa melhor a estrutura do código do programa do que a árvore de sintaxe. Uma AST pode ser editada e aprimorada com informações como propriedades e anotações para cada elemento que ela contém. Seu objetivo neste terceiro projeto é transformar a árvore de sintaxe em uma AST.

## Nós de uma Árvore de Sintaxe Abstrata

Esta seção define classes para diferentes tipos de nós de uma AST. Durante a análise sintática, você criará esses nós e os conectará. Em geral, você terá um nó AST diferente para cada tipo de regra gramatical.

A classe `Node` abaixo deve ser usada como base para implementar os diferentes tipos de nós da AST.

```python
class Node(object):
    """
    Base class example for the AST nodes.

    By default, instances of classes have a dictionary for attribute storage.
    This wastes space for objects having very few instance variables.
    The space consumption can become acute when creating large numbers of instances.

    The default can be overridden by defining __slots__ in a class definition.
    The __slots__ declaration takes a sequence of instance variables and reserves
    just enough space in each instance to hold a value for each variable.
    Space is saved because __dict__ is not created for each instance.
    """
    __slots__ = ("coord", "attrs",)

    def __init__(self, coord=None):
        self.coord = coord
        self.attrs = {}

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    attr_names = ()
```

Para cada um dos nós AST específicos, você precisa adicionar a especificação `__slots__` apropriada que indica quais campos devem ser armazenados:

```python
class Program(Node):
    
    __slots__ = ("stmts",)

    def __init__(self, stmts, coord=None):
        super().__init__(coord)
        self.stmts = stmts

    def children(self):
        nodelist = []
        for i, child in enumerate(self.stmts or []):
            nodelist.append(('stmts[%d]' % i, child))
        return tuple(nodelist)
```

Apenas como outro exemplo, para um operador binário, você pode armazenar o operador, a expressão esquerda e a expressão direita assim:

```python
class BinaryOp(Node):
    
    __slots__ = ("op", "left", "right",)

    def __init__(self, op, left, right, coord=None):
        super().__init__(coord)
        self.op = op
        self.left = left
        self.right = right

    def children(self):
        nodelist = []
        if self.left is not None:
            nodelist.append(('left', self.left))
        if self.right is not None:
            nodelist.append(('right', self.right))
        return tuple(nodelist)

    attr_names = ("op",)
```

Para uma constante, você pode armazenar o tipo e o valor, assim:

```python
class Literal(Node):
    
    __slots__ = ("type", "value",)

    def __init__(self, type, value, coord=None):
        super().__init__(coord)
        self.type = type
        self.value = value

    def children(self):
        return ()

    attr_names = ("type", "value",)
```

Os campos especificados em `__slots__` que não forem nós filhos retornados no método `children()` , mas sim atributos do próprio nó que serão exibidos quando ele for impresso, devem ter seus nomes indicados em `attr_names`.

**Sugestão:** Você deve começar de forma simples e trabalhar incrementalmente até construir a gramática completa.

## Classes de Nós AST

A lista abaixo define as classes de nós AST e os nomes de atributos esperados que devem ser usados no analisador sintático:

BinaryOp (op), BreakStatement ( ), ChuckOp ( ), ContinueStatement ( ), ExpressionAsStatement ( ), ExprList ( ), ID (name), IfStatement ( ), Literal (type, value), Location (name), PrintStatement ( ), Program ( ), StmtList ( ), Type (name), UnaryOp (op), VarDecl (name), WhileStatement ( ).

## Mostrando a AST

Considere como exemplo o seguinte programa em uChuck:

```
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}
```

A impressão da AST para o exemplo acima é:

```
Program:
    ExpressionAsStatement:
        ChuckOp: @ 2:1
            VarDecl: ID(name=n)
                Type: int @ 2:6
            Literal: int, 1 @ 2:1
    ExpressionAsStatement:
        ChuckOp: @ 3:1
            VarDecl: ID(name=value)
                Type: int @ 3:6
            Literal: int, 1 @ 3:1
    WhileStatement: @ 5:1
        BinaryOp: < @ 5:8
            Location: n @ 5:8
            Literal: int, 10 @ 5:12
        StmtList: @ 6:1
            ExpressionAsStatement:
                ChuckOp: @ 7:2
                    Location: value @ 7:15
                    BinaryOp: * @ 7:2
                        Location: value @ 7:2
                        Location: n @ 7:10
            ExpressionAsStatement:
                PrintStatement: @ 8:2
                    Location: value @ 8:6
            ExpressionAsStatement:
                ChuckOp: @ 9:2
                    Location: n @ 9:11
                    BinaryOp: + @ 9:2
                        Location: n @ 9:2
                        Literal: int, 1 @ 9:6
```

Os métodos para gerar uma representação textual dos nós AST e imprimir todos os seus atributos estão mostrados abaixo:

In [1]:
import sys

def represent_node(obj, indent):
    def _repr(obj, indent, printed_set):
        """
        Get the representation of an object, with dedicated pprint-like format for lists.
        """
        if isinstance(obj, list):
            indent += 1
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            return (
                "["
                + (sep.join((_repr(e, indent, printed_set) for e in obj)))
                + final_sep
                + "]"
            )
        elif isinstance(obj, Node):
            if obj in printed_set:
                return ""
            else:
                printed_set.add(obj)
            result = obj.__class__.__name__ + "("
            indent += len(obj.__class__.__name__) + 1
            attrs = []
            for name in obj.__slots__:
                if name == "bind":
                    continue
                value = getattr(obj, name)
                value_str = _repr(value, indent + len(name) + 1, printed_set)
                attrs.append(name + "=" + value_str)
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            result += sep.join(attrs)
            result += ")"
            return result
        elif isinstance(obj, str):
            return obj
        else:
            return ""

    # avoid infinite recursion with printed_set
    printed_set = set()
    return _repr(obj, indent, printed_set)

class Node:
    """Abstract base class for AST nodes."""

    __slots__ = ("coord", "attrs",)

    def __init__(self, coord=None):
        self.coord = coord
        self.attrs = {}

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    attr_names = ()

    def __repr__(self):
        """Generates a python representation of the current node"""
        return represent_node(self, 0)

    def show(
        self,
        buf=sys.stdout,
        offset=0,
        attrnames=False,
        nodenames=False,
        showcoord=False,
        _my_node_name=None,
    ):
        """Pretty print the Node and all its attributes and children (recursively) to a buffer.
        buf:
            Open IO buffer into which the Node is printed.
        offset:
            Initial offset (amount of leading spaces)
        attrnames:
            True if you want to see the attribute names in name=value pairs. False to only see the values.
        nodenames:
            True if you want to see the actual node names within their parents.
        showcoord:
            Do you want the coordinates of each Node to be displayed.
        """
        lead = " " * offset
        if nodenames and _my_node_name is not None:
            buf.write(lead + self.__class__.__name__ + " <" + _my_node_name + ">: ")
            inner_offset = len(self.__class__.__name__ + " <" + _my_node_name + ">: ")
        else:
            buf.write(lead + self.__class__.__name__ + ":")
            inner_offset = len(self.__class__.__name__ + ":")

        if self.attr_names:
            if attrnames:
                nvlist = [
                    (n, represent_node(getattr(self, n), offset+inner_offset+1+len(n)+1))
                    for n in self.attr_names
                    if getattr(self, n) is not None
                ]
                attrstr = ", ".join("%s=%s" % nv for nv in nvlist)
            else:
                vlist = [getattr(self, n) for n in self.attr_names]
                attrstr = ", ".join(
                    represent_node(v, offset + inner_offset + 1) for v in vlist
                )
            buf.write(" " + attrstr)

        if showcoord:
            if self.coord and self.coord.line != 0:
                buf.write(" %s" % self.coord)
        buf.write("\n")

        for (child_name, child) in self.children():
            child.show(buf, offset + 4, attrnames, nodenames, showcoord, child_name)

## Lidando com informações de linha e coluna na AST

A classe `Coord` abaixo deve ser usada para armazenar (e mostrar na AST) as linhas e colunas das produções no código fonte.

In [2]:
class Coord:
    """Coordinates of a syntactic element. Consists of:
    - Line number
    - (optional) column number, for the Lexer
    """

    __slots__ = ("line", "column")

    def __init__(self, line, column=None):
        self.line = line
        self.column = column

    def __str__(self):
        if self.line and self.column is not None:
            coord_str = "@ %s:%s" % (self.line, self.column)
        elif self.line:
            coord_str = "@ %s" % (self.line)
        else:
            coord_str = ""
        return coord_str

Para capturar as coordenadas do símbolo terminal mais à esquerda de uma produção, use o seguinte código na classe `UChuckParser` (a coordenada inclui os números de linha e de coluna e ambos seguem a semântica do analisador léxico, começando em 1):

```python
    def _token_coord(self, p):
        line, column = self.lexer._make_location(p)
        return Coord(line, column)
```

## Esboço da Construção da AST

Complete o código abaixo, escrevendo classes de nós AST para cada construção da linguagem uChuck.

In [3]:
import sys

def represent_node(obj, indent):
    def _repr(obj, indent, printed_set):
        """
        Get the representation of an object, with dedicated pprint-like format for lists.
        """
        if isinstance(obj, list):
            indent += 1
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            return (
                "["
                + (sep.join((_repr(e, indent, printed_set) for e in obj)))
                + final_sep
                + "]"
            )
        elif isinstance(obj, Node):
            if obj in printed_set:
                return ""
            else:
                printed_set.add(obj)
            result = obj.__class__.__name__ + "("
            indent += len(obj.__class__.__name__) + 1
            attrs = []
            for name in obj.__slots__:
                if name == "bind":
                    continue
                value = getattr(obj, name)
                value_str = _repr(value, indent + len(name) + 1, printed_set)
                attrs.append(name + "=" + value_str)
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            result += sep.join(attrs)
            result += ")"
            return result
        elif isinstance(obj, str):
            return obj
        else:
            return ""

    # avoid infinite recursion with printed_set
    printed_set = set()
    return _repr(obj, indent, printed_set)

class Coord:
    """Coordinates of a syntactic element. Consists of:
    - Line number
    - (optional) column number, for the Lexer
    """

    __slots__ = ("line", "column")

    def __init__(self, line, column=None):
        self.line = line
        self.column = column

    def __str__(self):
        if self.line and self.column is not None:
            coord_str = "@ %s:%s" % (self.line, self.column)
        elif self.line:
            coord_str = "@ %s" % (self.line)
        else:
            coord_str = ""
        return coord_str

class Node:
    """Abstract base class for AST nodes."""

    __slots__ = ("coord", "attrs",)

    def __init__(self, coord=None):
        self.coord = coord
        self.attrs = {}

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    attr_names = ()

    def __repr__(self):
        """Generates a python representation of the current node"""
        return represent_node(self, 0)

    def show(
        self,
        buf=sys.stdout,
        offset=0,
        attrnames=False,
        nodenames=False,
        showcoord=False,
        _my_node_name=None,
    ):
        """Pretty print the Node and all its attributes and children (recursively) to a buffer.
        buf:
            Open IO buffer into which the Node is printed.
        offset:
            Initial offset (amount of leading spaces)
        attrnames:
            True if you want to see the attribute names in name=value pairs. False to only see the values.
        nodenames:
            True if you want to see the actual node names within their parents.
        showcoord:
            Do you want the coordinates of each Node to be displayed.
        """
        lead = " " * offset
        if nodenames and _my_node_name is not None:
            buf.write(lead + self.__class__.__name__ + " <" + _my_node_name + ">: ")
            inner_offset = len(self.__class__.__name__ + " <" + _my_node_name + ">: ")
        else:
            buf.write(lead + self.__class__.__name__ + ":")
            inner_offset = len(self.__class__.__name__ + ":")

        if self.attr_names:
            if attrnames:
                nvlist = [
                    (n, represent_node(getattr(self, n), offset+inner_offset+1+len(n)+1))
                    for n in self.attr_names
                    if getattr(self, n) is not None
                ]
                attrstr = ", ".join("%s=%s" % nv for nv in nvlist)
            else:
                vlist = [getattr(self, n) for n in self.attr_names]
                attrstr = ", ".join(
                    represent_node(v, offset + inner_offset + 1) for v in vlist
                )
            buf.write(" " + attrstr)

        if showcoord:
            if self.coord and self.coord.line != 0:
                buf.write(" %s" % self.coord)
        buf.write("\n")

        for (child_name, child) in self.children():
            child.show(buf, offset + 4, attrnames, nodenames, showcoord, child_name)

class NodeVisitor:
    """ A base NodeVisitor class for visiting uchuck_ast nodes.
        Subclass it and define your own visit_XXX methods, where
        XXX is the class name you want to visit with these
        methods.
    """

    _method_cache = None

    def visit(self, node):
        """ Visit a node.
        """

        if self._method_cache is None:
            self._method_cache = {}

        visitor = self._method_cache.get(node.__class__.__name__, None)
        if visitor is None:
            method = 'visit_' + node.__class__.__name__
            visitor = getattr(self, method, self.generic_visit)
            self._method_cache[node.__class__.__name__] = visitor

        return visitor(node)

    def generic_visit(self, node):
        """ Called if no explicit visitor function exists for a
            node. Implements preorder visiting of the node.
        """
        for _, child in node.children():
            self.visit(child)

class BinaryOp(Node):

    __slots__ = ("op", "left", "right",)

    def __init__(self, op, left, right, coord=None):
        super().__init__(coord)
        self.op = op
        self.left = left
        self.right = right

    def children(self):
        nodelist = []
        if self.left is not None:
            nodelist.append(('left', self.left))
        if self.right is not None:
            nodelist.append(('right', self.right))
        return tuple(nodelist)

    attr_names = ("op",)

class BreakStatement(Node):
    __slots__ = ()

    def __init__(self, coord=None):
        super().__init__(coord)

    def children(self):
        return ()

    attr_names = ()

class ChuckOp(Node):
    __slots__ = ("left", "right")

    def __init__(self, left, right, coord=None):
        super().__init__(coord)
        self.left = left
        self.right = right

    def children(self):
        return (('right', self.right), ('left', self.left))

    attr_names = ()

class ContinueStatement(Node):
    __slots__ = ()

    def __init__(self, coord=None):
        super().__init__(coord)

    def children(self):
        return ()

    attr_names = ()

class ExpressionAsStatement(Node):
    __slots__ = ("expression",)

    def __init__(self, expression, coord=None):
        super().__init__(coord)
        self.expression = expression

    def children(self):
        if self.expression is not None:
            return (('expression', self.expression),)
        return ()

    attr_names = ()

class ExprList(Node):
    __slots__ = ("exprs",)

    def __init__(self, exprs, coord=None):
        super().__init__(coord)
        self.exprs = exprs

    def children(self):
        nodelist = []
        for i, child in enumerate(self.exprs or []):
            nodelist.append(('exprs[%d]' % i, child))
        return tuple(nodelist)

    attr_names = ()

class ID(Node):
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name

    def children(self):
        return ()

    attr_names = ("name",)

class IfStatement(Node):
    __slots__ = ("test", "consequence", "alternative")

    def __init__(self, test, consequence, alternative, coord=None):
        super().__init__(coord)
        self.test = test
        self.consequence = consequence
        self.alternative = alternative

    def children(self):
        nodelist = []
        if self.test is not None:
            nodelist.append(('test', self.test))
        if self.consequence is not None:
            nodelist.append(('consequence', self.consequence))
        if self.alternative is not None:
            nodelist.append(('alternative', self.alternative))
        return tuple(nodelist)

    attr_names = ()

class Literal(Node):

    __slots__ = ("type", "value",)

    def __init__(self, type, value, coord=None):
        super().__init__(coord)
        self.type = type
        self.value = value

    def children(self):
        return ()

    attr_names = ("type", "value",)

class Location(Node):
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name

    def children(self):
        return ()

    attr_names = ("name",)

class PrintStatement(Node):
    __slots__ = ("expression",)

    def __init__(self, expression, coord=None):
        super().__init__(coord)
        self.expression = expression

    def children(self):
        if self.expression is not None:
            return (('expression', self.expression),)
        return ()

    attr_names = ()

class Program(Node):

    __slots__ = ("stmts",)

    def __init__(self, stmts, coord=None):
        super().__init__(coord)
        self.stmts = stmts

    def children(self):
        nodelist = []
        for i, child in enumerate(self.stmts or []):
            nodelist.append(('stmts[%d]' % i, child))
        return tuple(nodelist)

class StmtList(Node):
    __slots__ = ("stmts",)

    def __init__(self, stmts, coord=None):
        super().__init__(coord)
        self.stmts = stmts

    def children(self):
        nodelist = []
        for i, child in enumerate(self.stmts or []):
            nodelist.append(('stmts[%d]' % i, child))
        return tuple(nodelist)

    attr_names = ()

class Type(Node):
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name

    def children(self):
        return ()

    attr_names = ("name",)

class UnaryOp(Node):
    __slots__ = ("op", "operand")

    def __init__(self, op, operand, coord=None):
        super().__init__(coord)
        self.op = op
        self.operand = operand

    def children(self):
        nodelist = []
        if self.operand is not None:
            nodelist.append(('operand', self.operand))
        return tuple(nodelist)

    attr_names = ("op",)

class VarDecl(Node):
    __slots__ = ("dtype", "name")

    def __init__(self, dtype, name, coord=None):
        super().__init__(coord)
        self.dtype = dtype
        self.name = name

    def children(self):
        nodelist = []
        if self.dtype is not None:
            nodelist.append(('dtype', self.dtype))
        return tuple(nodelist)

    attr_names = ("name",)

class WhileStatement(Node):
    __slots__ = ("test", "body")

    def __init__(self, test, body, coord=None):
        super().__init__(coord)
        self.test = test
        self.body = body

    def children(self):
        nodelist = []
        if self.test is not None:
            nodelist.append(('test', self.test))
        if self.body is not None:
            nodelist.append(('body', self.body))
        return tuple(nodelist)

    attr_names = ()

In [4]:
!pip install sly
from sly import Lexer, Parser

Copie o código da classe `UChuckLexer` que você escreveu no [primeiro projeto](https://colab.research.google.com/drive/1FXsbf3mp9rf-Cce-c9qk_8S43ppcPd4k?usp=sharing) e cole na célula abaixo.

In [5]:
class UChuckLexer(Lexer):
    """A lexer for the uChuck language."""

    def __init__(self, error_func):
        """Create a new Lexer.
        An error function. Will be called with an error
        message, line and column as arguments, in case of
        an error during lexing.
        """
        self.error_func = error_func

    # Reserved keywords
    keywords = {
        'int': "INT",
        'float': "FLOAT",
        'if': "IF",
        'else': "ELSE",
        'while': "WHILE",
        'break': "BREAK",
        'continue': "CONTINUE",
        'true': "TRUE",
        'false': "FALSE",
    }

    # All the tokens recognized by the lexer
    tokens = tuple(keywords.values()) + (
        # Identifiers
        "ID",
        # Constants
        "FLOAT_VAL",
        "INT_VAL",
        "STRING_LIT",
        # Operators
        "PLUS",
        "MINUS",
        "TIMES",
        "DIVIDE",
        "PERCENT",
        "LE",
        "LT",
        "GE",
        "GT",
        "EQ",
        "NEQ",
        "AND",
        "OR",
        "EXCLAMATION",
        # Assignment
        "CHUCK",
        # Delimeters
        "LPAREN",
        "RPAREN",  # ( )
        "LBRACE",
        "RBRACE",  # { }
        "L_HACK",
        "R_HACK",  # <<< >>>
        "COMMA",
        "SEMI",  # , ;
    )

    # String containing ignored characters (between tokens)
    ignore = " \t"

    # Other ignored patterns
    ignore_newline = r'\n+'
    ignore_comment = r'/\*(.|\n)*?\*/|//.*'

    # Regular expression rules for tokens
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    FLOAT_VAL = r'\d+\.\d+|\d+\.\d*|\.\d+'
    INT_VAL = r'\d+'
    STRING_LIT = r'"([^"\\]|\\.)*"'
    
    unterm_string = r'"(\\["\\]|[^"\\])*'
    unterm_comment = r'/\*.*'

    L_HACK = r'<<<'
    R_HACK = r'>>>'

    LE = r'<='
    GE = r'>='
    EQ = r'=='
    NEQ = r'!='
    AND = r'&&'
    OR = r'\|\|'
    CHUCK = r'=>'

    LT = r'<'
    GT = r'>'
    PLUS = r'\+'
    MINUS = r'-'
    TIMES = r'\*'
    DIVIDE = r'/'
    PERCENT = r'%'
    EXCLAMATION = r'!'

    LPAREN = r'\('
    RPAREN = r'\)'
    LBRACE = r'\{'
    RBRACE = r'\}'
    COMMA = r','
    SEMI = r';'

    # Special cases
    def ID(self, t):
      t.type = self.keywords.get(t.value, "ID")
      return t

    # Define a rule so we can track line numbers
    def ignore_newline(self, t):
        self.lineno += len(t.value)

    def ignore_comment(self, t):
        self.lineno += t.value.count("\n")

    def find_column(self, token):
        """Find the column of the token in its line."""
        last_cr = self.text.rfind('\n', 0, token.index)
        return token.index - last_cr

    # Internal auxiliary methods
    def _error(self, msg, token):
        location = self._make_location(token)
        self.error_func(msg, location[0], location[1])
        self.index += 1

    def _make_location(self, token):
        return token.lineno, self.find_column(token)

    def unterm_comment(self, t):
        msg = "Unterminated comment"
        self._error(msg, t)

    def unterm_string(self, t):
        msg = "Unterminated string literal"
        self._error(msg, t)

    def error(self, t):
        msg = "Illegal character %s" % repr(t.value[0])
        self._error(msg, t)

    # Scanner (used only for test)
    def scan(self, text):
        output = ""
        for tok in self.tokenize(text):
            print(tok)
            output += str(tok) + "\n"
        return output

Use o código da classe `UChuckParser` que você escreveu no [segundo projeto](https://colab.research.google.com/drive/10vT_BqOi8F4nVUGucBeXVl1nH6zTMLJw?usp=sharing) como base para completar completar o código abaixo:
- Defina as regras de precedência e associatividade conforme a implementação que você escreveu no [segundo projeto](https://colab.research.google.com/drive/10vT_BqOi8F4nVUGucBeXVl1nH6zTMLJw?usp=sharing).
- Complete o código dos métodos referentes às regras gramaticas usando a implementação do [segundo projeto](https://colab.research.google.com/drive/10vT_BqOi8F4nVUGucBeXVl1nH6zTMLJw?usp=sharing).
- Modifique as regras para retornarem nós AST ao invés de tuplas e criando nós AST para identificadores e constantes.

In [6]:
class UChuckParser(Parser):
    """A parser for the uChuck language."""

    # Get the token list from the lexer (required)
    tokens = UChuckLexer.tokens

    precedence = (
        ('left', 'COMMA'),
        ('left', 'CHUCK'),
        ('left', 'OR'),
        ('left', 'AND'),
        ('left', 'EQ', 'NEQ'),
        ('left', 'LT', 'LE', 'GT', 'GE'),
        ('left', 'PLUS', 'MINUS'),
        ('left', 'TIMES', 'DIVIDE', 'PERCENT'),
        ('right', 'EXCLAMATION'),
    )

    def __init__(self, error_func=lambda msg, x, y: print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)):
        self.lexer = UChuckLexer(error_func)

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    def _token_coord(self, p):
        return Coord(p.lineno, self.lexer.find_column(p))

    def error(self, p):
        if p:
            if hasattr(p, 'lineno'):
                print("Error at line %d near the symbol %s " % (p.lineno, p.value))
            else:
                print("Error near the symbol %s" % p.value)
        else:
            print("Error at the end of input")

    # <program> ::= <statement_list> EOF
    @_('statement_list')
    def program(self, p):
        return Program(p.statement_list)

    # <statement_list> ::= { <statement> }+
    @_('statement statement_list')
    def statement_list(self, p):
        return [p.statement] + p.statement_list

    @_('statement')
    def statement_list(self, p):
        return [p.statement]

    # <statement> ::= <expression_statement>
    #               | <loop_statement>
    #               | <selection_statement>
    #               | <jump_statement>
    #               | <code_segment>
    @_('expression_statement',
       'loop_statement',
       'selection_statement',
       'jump_statement',
       'code_segment')
    def statement(self, p):
        return p[0]

    # <jump_statement> ::= "break" ";"
    #                    | "continue" ";"
    @_('BREAK SEMI')
    def jump_statement(self, p):
        return BreakStatement(coord=self._token_coord(p))

    @_('CONTINUE SEMI')
    def jump_statement(self, p):
        return ContinueStatement(coord=self._token_coord(p))

    # <selection_statement> ::= "if" "(" <expression> ")" <statement> { "else" <statement> }?
    @_('IF LPAREN expression RPAREN statement ELSE statement')
    def selection_statement(self, p):
        return IfStatement(test=p.expression, consequence=p.statement0, alternative=p.statement1, coord=self._token_coord(p))

    @_('IF LPAREN expression RPAREN statement')
    def selection_statement(self, p):
        return IfStatement(test=p.expression, consequence=p.statement, alternative=None, coord=self._token_coord(p))

    # <loop_statement> ::= "while" "(" <expression> ")" <statement>
    @_('WHILE LPAREN expression RPAREN statement')
    def loop_statement(self, p):
        return WhileStatement(test=p.expression, body=p.statement, coord=self._token_coord(p))

    # <code_segment> ::= "{" { <statement_list> }? "}"
    @_('LBRACE statement_list RBRACE')
    def code_segment(self, p):
        return StmtList(stmts=p.statement_list, coord=self._token_coord(p))

    @_('LBRACE RBRACE')
    def code_segment(self, p):
        return StmtList(stmts=[], coord=self._token_coord(p))

    # <expression_statement> ::= { <expression> }? ";"
    @_('expression SEMI')
    def expression_statement(self, p):
        return ExpressionAsStatement(expression=p.expression, coord=None)

    @_('SEMI')
    def expression_statement(self, p):
        return ExpressionAsStatement(None, coord=None)

    # <expression> ::= <chuck_expression> { "," <chuck_expression> }*
    @_('expression COMMA expression')
    def expression(self, p):
        if isinstance(p.expression0, ExprList):
            expressions = p.expression0.exprs + [p.expression1]
            coord = p.expression0.coord
        else:
            expressions = [p.expression0, p.expression1]
            coord = p.expression0.coord
        
        return ExprList(exprs=expressions, coord=coord)

    @_('chuck_expression')
    def expression(self, p):
        return p.chuck_expression

    # <chuck_expression> ::= { <chuck_expression> "=>" }? <decl_expression>
    @_('chuck_expression CHUCK decl_expression')
    def chuck_expression(self, p):
        return ChuckOp(left=p.chuck_expression, right=p.decl_expression, coord=self._token_coord(p))
    
    @_('decl_expression')
    def chuck_expression(self, p):
        return p.decl_expression

    # <decl_expression> ::= <binary_expression>
    #                     | <type_decl> <identifier>
    @_('binary_expression')
    def decl_expression(self, p):
        return p.binary_expression

    @_('type_decl identifier')
    def decl_expression(self, p):
        return VarDecl(dtype=p.type_decl, name=ID(name=p.identifier.name, coord=p.identifier.coord), coord=None)

    # <type_decl> ::= "int"
    #               | "float"
    #               | <identifier>
    @_('INT')
    def type_decl(self, p):
        return Type(name='int', coord=self._token_coord(p))

    @_('FLOAT')
    def type_decl(self, p):
        return Type(name='float', coord=self._token_coord(p))

    @_('identifier')
    def type_decl(self, p):
        return Type(name=p.identifier.name, coord=p.identifier.coord)

    # <binary_expression> ::= <unary_expression>
    #                       | <binary_expression> "+"  <binary_expression>
    #                       | <binary_expression> "-"  <binary_expression>
    #                       | <binary_expression> "*"  <binary_expression>
    #                       | <binary_expression> "/"  <binary_expression>
    #                       | <binary_expression> "%"  <binary_expression>
    #                       | <binary_expression> "<=" <binary_expression>
    #                       | <binary_expression> "<"  <binary_expression>
    #                       | <binary_expression> ">=" <binary_expression>
    #                       | <binary_expression> ">"  <binary_expression>
    #                       | <binary_expression> "==" <binary_expression>
    #                       | <binary_expression> "!=" <binary_expression>
    #                       | <binary_expression> "&&" <binary_expression>
    #                       | <binary_expression> "||" <binary_expression>
    @_('binary_expression PLUS binary_expression',
       'binary_expression MINUS binary_expression',
       'binary_expression TIMES binary_expression',
       'binary_expression DIVIDE binary_expression',
       'binary_expression PERCENT binary_expression',
       'binary_expression LE binary_expression',
       'binary_expression LT binary_expression',
       'binary_expression GE binary_expression',
       'binary_expression GT binary_expression',
       'binary_expression EQ binary_expression',
       'binary_expression NEQ binary_expression',
       'binary_expression AND binary_expression',
       'binary_expression OR binary_expression',
       )
    def binary_expression(self, p):
        return BinaryOp(op=p[1], left=p.binary_expression0, right=p.binary_expression1, coord=p.binary_expression0.coord)
    
    @_('unary_expression')
    def binary_expression(self, p):
        return p.unary_expression

    # <unary_expression> ::= <primary_expression>
    #                      | <unary_operator> <unary_expression>
    @_('unary_operator unary_expression')
    def unary_expression(self, p):
        return UnaryOp(op=p.unary_operator, operand=p.unary_expression, coord=p.unary_expression.coord)

    @_('MINUS expression')
    def expression(self, p):
        return UnaryOp('-', p.expression, coord=self._token_coord(p.MINUS))

    @_('primary_expression')
    def unary_expression(self, p):
        return p.primary_expression

    # <unary_operator> ::= "+"
    #                    | "!"
    @_('PLUS')
    def unary_operator(self, p):
        return '+'
    
    @_('MINUS')
    def unary_operator(self, p):
        return '-'

    @_('EXCLAMATION')
    def unary_operator(self, p):
        return '!'

    # <primary_expression> ::= <literal>
    #                        | <location>
    #                        | "<<<" <expression> ">>>"
    #                        | "(" <expression> ")"
    @_('literal')
    def primary_expression(self, p):
        return p.literal

    @_('location')
    def primary_expression(self, p):
        return p.location

    @_('L_HACK expression R_HACK')
    def primary_expression(self, p):
        return PrintStatement(expression=p.expression, coord=self._token_coord(p))

    @_('LPAREN expression RPAREN')
    def primary_expression(self, p):
        return p.expression

    @_('INT_VAL')
    def literal(self, p):
        return Literal(type='int', value=p.INT_VAL, coord=self._token_coord(p))
    
    @_('FLOAT_VAL')
    def literal(self, p):
        return Literal(type='float', value=p.FLOAT_VAL, coord=self._token_coord(p))
    
    @_('STRING_LIT')
    def literal(self, p):
        return Literal(type='string', value=p.STRING_LIT, coord=self._token_coord(p))
    
    @_('TRUE')
    def literal(self, p):
        return Literal(type='int', value='1', coord=self._token_coord(p))

    @_('FALSE')
    def literal(self, p):
        return Literal(type='int', value='0', coord=self._token_coord(p))

    # <location> ::= <identifier>
    @_('identifier')
    def location(self, p):
        return Location(name=p.identifier.name, coord=p.identifier.coord)

    # <identifier> ::= ID
    @_('ID')
    def identifier(self, p):
        return ID(name=p.ID, coord=Coord(p.lineno, self.lexer.find_column(p)))

In [7]:
def print_error(msg, x, y):
    # use stdout to match with the output in the .out test files
    print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)

def main(args):
    parser = UChuckParser(print_error)
    with open(args[0], 'r') if len(args) > 0 else sys.stdin as f:
        ast = parser.parse(f.read())
        if ast is not None:
            ast.show(showcoord=True)

## Teste
Para o desenvolvimento inicial, tente executar o analisador sintático em um arquivo de entrada de exemplo, como:

```
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}
```

In [8]:
%%file test.uck
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}

Overwriting test.uck


E o resultado será semelhante ao texto mostrado abaixo.

```
Program:
    ExpressionAsStatement:
        ChuckOp: @ 2:1
            VarDecl: ID(name=n)
                Type: int @ 2:6
            Literal: int, 1 @ 2:1
    ExpressionAsStatement:
        ChuckOp: @ 3:1
            VarDecl: ID(name=value)
                Type: int @ 3:6
            Literal: int, 1 @ 3:1
    WhileStatement: @ 5:1
        BinaryOp: < @ 5:8
            Location: n @ 5:8
            Literal: int, 10 @ 5:12
        StmtList: @ 6:1
            ExpressionAsStatement:
                ChuckOp: @ 7:2
                    Location: value @ 7:15
                    BinaryOp: * @ 7:2
                        Location: value @ 7:2
                        Location: n @ 7:10
            ExpressionAsStatement:
                PrintStatement: @ 8:2
                    Location: value @ 8:6
            ExpressionAsStatement:
                ChuckOp: @ 9:2
                    Location: n @ 9:11
                    BinaryOp: + @ 9:2
                        Location: n @ 9:2
                        Literal: int, 1 @ 9:6
```

In [9]:
main(["test.uck"])

Program:
    ExpressionAsStatement:
        ChuckOp: @ 2:1
            VarDecl: ID(name=n)
                Type: int @ 2:6
            Literal: int, 1 @ 2:1
    ExpressionAsStatement:
        ChuckOp: @ 3:1
            VarDecl: ID(name=value)
                Type: int @ 3:6
            Literal: int, 1 @ 3:1
    WhileStatement: @ 5:1
        BinaryOp: < @ 5:8
            Location: n @ 5:8
            Literal: int, 10 @ 5:12
        StmtList: @ 6:1
            ExpressionAsStatement:
                ChuckOp: @ 7:2
                    Location: value @ 7:15
                    BinaryOp: * @ 7:2
                        Location: value @ 7:2
                        Location: n @ 7:10
            ExpressionAsStatement:
                PrintStatement: @ 8:2
                    Location: value @ 8:6
            ExpressionAsStatement:
                ChuckOp: @ 9:2
                    Location: n @ 9:11
                    BinaryOp: + @ 9:2
                        Location: n @ 9:2
             

## Anexo

A lista abaixo define os campos que devem ser armazenados em cada tipo de nó AST:

```
###########################################################
## Nós do uChuck ##########################################
###########################################################

#   <name>*     - um nó filho
#   <name>**    - uma sequência de nós filhos
#   <name>      - um atributo

BinaryOp: [op, left*, right*]

BreakStatement: []

ChuckOp: [location*, expression*]

ContinueStatement: []

ExpressionAsStatement: [expression*]

ExprList: [exprs**]

ID: [name]

IfStatement: [test*, consequence*, alternative*]

Literal: [type, value]

Location: [name]

PrintStatement: [expression*]

Program: [stmts**]

StmtList: [stmts**]

Type: [name]

UnaryOp: [op, operand*]

VarDecl: [dtype*, name]

WhileStatement: [test*, body*]
```

A lista abaixo define os nós AST que devem ser retornados em cada regra da gramática:

```
program = Program(statement_list)

statement_list = list(statement)

statement = expression_statement
          | loop_statement
          | selection_statement
          | jump_statement
          | code_segment

jump_statement = BreakStatement(coord=(lineno, column))
               | ContinueStatement(coord=(lineno, column))

selection_statement = IfStatement(expression, statement0, statement1, coord=(lineno, column))

loop_statement = WhileStatement(expression, statement, coord=(lineno, column))

code_segment = StmtList(statement_list, coord=(lineno, column))

expression_statement = ExpressionAsStatement(expression)

expression = chuck_expression
           | ExprList(list(chuck_expression), coord=p.expression.coord)

chuck_expression = ChuckOp(decl_expression, chuck_expression, coord=(lineno, column))
                 | decl_expression

decl_expression = binary_expression
                | VarDecl(type_decl, ID(str(ID), coord=(lineno, column)))

type_decl = Type('int', coord=(lineno, column))
          | Type('float', coord=(lineno, column))
          | Type(str(ID), coord=(lineno, column))

binary_expression = unary_expression
                  | BinaryOp('+', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('-', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('*', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('/', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('%', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('<=', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('<', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('>=', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('>', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('==', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('!=', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('&&', binary_expression0, binary_expression1, coord=(lineno, column))
                  | BinaryOp('||', binary_expression0, binary_expression1, coord=(lineno, column))

unary_expression = primary_expression
                 | UnaryOp(unary_operator, unary_expression, coord=unary_expression.coord)

unary_operator = '+'
               | '-'
               | '!'

primary_expression = literal
                   | location
                   | PrintStatement(expression, coord=(lineno, column))
                   | expression

literal = Literal('int', str(INT_VAL), coord=(lineno, column))
        | Literal('float', str(FLOAT_VAL), coord=(lineno, column))
        | Literal('string', str(STRING_LIT), coord=(lineno, column))
        | Literal('int', '1', coord=(lineno, column))
        | Literal('int', '0', coord=(lineno, column))

location = Location(str(ID), coord=(lineno, column))
```
